In [1]:
 ! pip install -q kaggle
 

In [2]:
 from google.colab import files

In [ ]:
files.upload()

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d stefanoleone992/imdb-extensive-dataset

 70% 58.0M/82.3M [00:00<00:00, 99.9MB/s]
100% 82.3M/82.3M [00:00<00:00, 185MB/s] 


In [9]:
! mkdir train

In [11]:
! unzip imdb-extensive-dataset.zip -d train

Archive:  imdb-extensive-dataset.zip
  inflating: train/IMDb movies.csv   
  inflating: train/IMDb names.csv    
  inflating: train/IMDb ratings.csv  
  inflating: train/IMDb title_principals.csv  


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
df = pd.read_csv("./train/IMDb movies.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df.shape

(85855, 22)

In [18]:
# sample our df

sampled_df = df.sample(n = 2021,random_state=1)


In [19]:
sampled_df.shape

(2021, 22)

In [20]:
# create the list of important columns
columns = ['actors','director','writer','genre','description','title']

In [21]:
simplified_df = sampled_df[columns]

In [22]:
# check for missing values
simplified_df[simplified_df.isna().any(axis=1)].shape

(87, 6)

In [23]:
simplified_df.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
simplified_df.shape

(1934, 6)

In [26]:
# recheck the null values
simplified_df.isna().any().any()

False

In [27]:
simplified_df.head(5)

,actors,director,writer,genre,description,title
4683,"Deanna Durbin, Gene Kelly, Richard Whorf, Dean...",Robert Siodmak,"W. Somerset Maugham, Herman J. Mankiewicz","Crime, Drama, Film-Noir",A young femme fatale-type woman realizes that ...,Vacanze a Natale
28176,"Arnold Schwarzenegger, Danny DeVito, Emma Thom...",Ivan Reitman,"Kevin Wade, Chris Conrad","Comedy, Romance, Sci-Fi","As part of a fertility research project, a mal...",Junior
16686,"Henning Jensen, Poul Reichhardt, Jens Okking, ...",Esben Høilund Carlsen,"Esben Høilund Carlsen, Torben Nielsen",Crime,A Danish man's girlfriend is killed by a drunk...,Nitten røde roser
79161,"Jamie Bernadette, Vanessa Reynolds, Stacey Dan...",Dylan Reynolds,Dylan Reynolds,"Action, Horror",A group of five women go camping in the woods ...,4/20 Massacre
40856,"Masatoshi Nagase, Akiko Monô, Tamaki Ogawa, Ku...",Hiroyuki Nakano,"Masatoshi Kimura, Hiroyuki Nakano","Comedy, Romance","Keisuke, an aspiring samurai-movie actor lands...",Stereo Future


In [70]:
# lets change the indices

simplified_df.index = range(simplified_df.shape[0])

In [71]:
simplified_df.head(5)

,actors,director,writer,genre,description,title
0,"Deanna Durbin, Gene Kelly, Richard Whorf, Dean...",Robert Siodmak,"W. Somerset Maugham, Herman J. Mankiewicz","Crime, Drama, Film-Noir",A young femme fatale-type woman realizes that ...,Vacanze a Natale
1,"Arnold Schwarzenegger, Danny DeVito, Emma Thom...",Ivan Reitman,"Kevin Wade, Chris Conrad","Comedy, Romance, Sci-Fi","As part of a fertility research project, a mal...",Junior
2,"Henning Jensen, Poul Reichhardt, Jens Okking, ...",Esben Høilund Carlsen,"Esben Høilund Carlsen, Torben Nielsen",Crime,A Danish man's girlfriend is killed by a drunk...,Nitten røde roser
3,"Jamie Bernadette, Vanessa Reynolds, Stacey Dan...",Dylan Reynolds,Dylan Reynolds,"Action, Horror",A group of five women go camping in the woods ...,4/20 Massacre
4,"Masatoshi Nagase, Akiko Monô, Tamaki Ogawa, Ku...",Hiroyuki Nakano,"Masatoshi Kimura, Hiroyuki Nakano","Comedy, Romance","Keisuke, an aspiring samurai-movie actor lands...",Stereo Future


In [72]:
# text to token counts generation
cv_actors = CountVectorizer().fit_transform(simplified_df['actors'])
cv_genre =  CountVectorizer().fit_transform(simplified_df['genre'])
cv_director =  CountVectorizer().fit_transform(simplified_df['director'])
cv_writer =  CountVectorizer().fit_transform(simplified_df['writer'])
cv_description =  CountVectorizer().fit_transform(simplified_df['description'])

In [29]:
cs_actors = cosine_similarity(cv_actors)
cs_genre = cosine_similarity(cv_genre)
cs_director = cosine_similarity(cv_director)
cs_writer = cosine_similarity(cv_writer)
cs_description = cosine_similarity(cv_description)


In [30]:
cs_actors

array([[1.        , 0.        , 0.        , ..., 0.        , 0.04652421,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.03806935, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.03806935, 0.        , ..., 1.        , 0.0725954 ,
        0.        ],
       [0.04652421, 0.        , 0.        , ..., 0.0725954 , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [31]:
cs_genre

array([[1.        , 0.        , 0.5       , ..., 0.35355339, 0.5       ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.5       ],
       [0.5       , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.35355339, 0.        , 0.        , ..., 1.        , 0.70710678,
        0.        ],
       [0.5       , 0.        , 0.        , ..., 0.70710678, 1.        ,
        0.        ],
       [0.        , 0.5       , 0.        , ..., 0.        , 0.        ,
        1.        ]])

lets do the statistics desciribing of our cs matrices, so that we can see whats the need to normalize our matrices

In [32]:
# method that describe the given matrix
def describe_ndarray(matrix):
  return pd.DataFrame(matrix).describe().mean(axis = 1)

In [33]:
describe_ndarray(cs_actors)

count    1934.000000
mean        0.009959
std         0.031347
min         0.000000
25%         0.000000
50%         0.000014
75%         0.011565
max         1.000000
dtype: float64

In [34]:
describe_ndarray(cs_genre)

count    1934.000000
mean        0.268967
std         0.280505
min         0.000000
25%         0.039770
50%         0.244341
75%         0.447862
max         1.000000
dtype: float64

## As we can see scale difference between the average values of two matrices, to make handpicking of 'weights' easier (in our case) we do normalization 


In [35]:
# function to normalize matrix

def normalize_2d(matrix):
    norm = np.linalg.norm(matrix)
    matrix = abs(matrix - matrix.mean())
    matrix = matrix/norm  # normalized matrix
    return matrix

In [36]:
normalized_cs_actors = normalize_2d(cs_actors)
normalized_cs_genre = normalize_2d(cs_genre)
normalized_cs_director = normalize_2d(cs_director)
normalized_cs_writer = normalize_2d(cs_writer)
normalized_cs_description = normalize_2d(cs_description)


In [37]:
describe_ndarray(normalized_cs_description)

count    1934.000000
mean        0.000226
std         0.000159
min         0.000001
25%         0.000103
50%         0.000203
75%         0.000326
max         0.002343
dtype: float64

In [38]:
describe_ndarray(normalized_cs_actors)

count    1934.000000
mean        0.000245
std         0.000418
min         0.000150
25%         0.000151
50%         0.000151
75%         0.000218
max         0.015028
dtype: float64

## Now we can see mean of values of matrix are in similar scale



### Now lets hand pick the values of weights for each feature 

In [40]:
# lets hand pick the weights for the features

w_actors, w_genre, w_director, w_description = 57.20, 86.00, 75.00, 44.00 

### get overall similarity by mulitplying each cosine similarity matrix with corresponding weights

In [41]:
# over-all similarity values

similarity = (normalized_cs_actors * w_actors)  + (normalized_cs_genre * w_genre) + (normalized_cs_director * w_director) + (normalized_cs_description * w_description)  

In [42]:
similarity.shape

(1934, 1934)

In [94]:
# method to get the index of the movie

def get_index_from_movie(movie):
  ind = simplified_df[simplified_df['title'] == movie].index
  if not ind.empty:
    return ind[0]
  else:
    return -1


In [106]:
# method to get the movie name from the list of indices

def get_movie_from_index(indices):
  movies = []

  for id in indices:
    movies.append(simplified_df.iloc[id]["title"])

  return movies

In [98]:
get_index_from_movie(movie1)

55

In [125]:
#method to get n similarity movies


def get_similar_movies(movie,n=5):
  # n --> count of similar movies
  
  # restrict max movies to 100
  if n > 100: n=100

  id = get_index_from_movie(movie)

  if id == -1:
    return ["No movie found!!"]
  
  # indices of top similarity scores
  # excluding first value (itself)
  indices = (-similarity[id]).argsort()[1:n+1]

  movies = get_movie_from_index(indices)

  return movies



In [126]:
# lets get the movies similar to these
movie1 = "Stella Days"
movie2 = "Highland Park"
movie3 = "Chastity"

In [116]:
print("\n\n".join(get_similar_movies(movie1,5)))

Secret Obsession

The Grief of Others

L'ultimo re di Scozia

I segreti di Filadelfia

Pollock


In [117]:
print("\n\n".join(get_similar_movies(movie2,5)))

D-Day Assassins

Magicians

Burden

MindGamers

Young and Dangerous 5


In [118]:
print("\n\n".join(get_similar_movies(movie3,5)))

I nostri ragazzi

Débarquement immédiat!

Taeter City

I familiari delle vittime non saranno avvertiti

Love Bites - Il morso dell'alba


In [131]:
get_similar_movies("movie that was not in 2021 movies we sampled")

['No movie found!!']